Using Ultralytics YOLOv8, ByteTrack and Supervision for tracking and counting
Detection with yolov8

Tracking with BoT-SORT

Counting using supervision

In [ ]:
!pip install ultralytics==8.0.106 supervision==0.14.0

In [ ]:
import ultralytics
ultralytics.__version__

WARNING ⚠️ Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'


'8.0.106'

In [ ]:
import supervision
supervision.__version__

'0.14.0'

In [ ]:
import torch
torch.__version__

'2.1.0+cu118'

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.track(source="d.mp4",conf=0.3, iou=0.5, save=True, tracker="bytetrack.yaml")

100%|██████████| 6.23M/6.23M [00:00<00:00, 65.8MB/s]


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/687) /content/d.mp4: 384x640 7 cars, 92.8ms
video 1/1 (2/687) /content/d.mp4: 384x640 7 cars, 9.6ms
video 1/1 (3/687)

In [ ]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture
cap = cv2.VideoCapture("d.mp4")

# Define the line coordinates
START = sv.Point(182, 254)
END = sv.Point(462, 254)


# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("d.mp4")
with sv.VideoSink("output_single_line.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if START.x < x < END.x and abs(y - START.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the line on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 384x640 7 cars, 109.7ms
Speed: 3.9ms preprocess, 109.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/track
<ipython-input-4-779eb8431fd1>:40: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections = sv.Detections.from_yolov8(results[0])

0: 384x640 7 cars, 6.5ms
Speed: 2.1ms preprocess, 6.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/track

0: 384x640 7 cars, 6.4ms
Speed: 1.7ms preprocess, 6.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/track

0: 384x640 7 cars, 6.3ms
Speed: 1.4ms preprocess, 6.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect

In [ ]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture
cap = cv2.VideoCapture("d.mp4")

# Define the line coordinates
START = sv.Point(182, 254)
END = sv.Point(462, 254)


# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("d.mp4")
with sv.VideoSink("output_single_line.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if START.x < x < END.x and abs(y - START.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the line on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 384x640 7 cars, 11.5ms
Speed: 1.8ms preprocess, 11.5ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/track4
<ipython-input-7-779eb8431fd1>:40: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections = sv.Detections.from_yolov8(results[0])

0: 384x640 7 cars, 9.0ms
Speed: 1.4ms preprocess, 9.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/track4

0: 384x640 7 cars, 9.3ms
Speed: 1.3ms preprocess, 9.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/track4

0: 384x640 7 cars, 9.3ms
Speed: 1.2ms preprocess, 9.3ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detec